# Laboratorio 7 (Aprendizaje por refuerzo) Grupo 1
 <h3>Implementacion de Algoritmos de Gradiente<h3>
 <HR>
 <h3>
  NOMBRES:
  
  * POLO ORELLANA BRAYAN SIMON
  * LOPEZ CHIRI DIMAR GABRIEL
  
  <br>
  CARRERA: INGENIERIA DE SISTEMAS
  <BR>
  FECHA: 17/06/2024 <BR>

  * [Enlace de invitacion para ser colaborador](https://github.com/bspoloo/SIS420-012024/invitations)
  
  * [Enlace al git hub](https://github.com/bspoloo/SIS420-012024/tree/main/Laboratorios/Laboratorio%207)

 <h3>

"Softmax Action Selection" o "Softmax Policy". En este enfoque, en lugar de seleccionar acciones basadas únicamente en sus valores estimados (como en los métodos $\epsilon$-greedy), se asigna una preferencia 
𝐻
𝑡
(
𝑎
)
H 
t
​
 (a) a cada acción 
𝑎
a. La probabilidad 
𝜋
𝑡
(
𝑎
)
π 
t
​
 (a) de seleccionar la acción 
𝑎
a en el tiempo 
𝑡
t está determinada por la función softmax:

$$
    \pi_t(a) = \frac{e^{H_t(a)}}{\sum_{b=1}^k e^{H_t(b)}}
$$

donde $\tilde{R_t}$ es el promedio de todas las recompensas hasta el instante $t$ (incluído). Este valor actúa como referencia con la que comparamos la recompensa. Si es mayor, la probabilidad de escoger esa acción en el futuro aumenta. Si es menor, la probabilidad se reduce. Las acciones no escogidas, se actualizan en el sentido opuesto.

$$
    \begin{array}{ll}
        H_{t+1}(a) = H_{t}(a) + \alpha(R_t-\tilde{R_t})(1-\pi_t(A_t)) & \text{y} \\
        H_{t+1}(a) = H_{t}(a) - \alpha(R_t-\tilde{R_t})\pi_t(a) & \text{para toda } a \neq A_t
    \end{array}
$$



## Importacion librerias
importamos las librerias necesarias para la realizacion de este laboratorio:

In [36]:

import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt#Importamos la librería gym para poder utilizar el entorno de OpenAI
import gymnasium as gym
#Importamos la librería numpy para poder trabajar con arrays
import numpy as np
#Importamos la librería matplotlib para poder realizar gráficas
import matplotlib.pyplot as plt 
#Importamos la librería os para poder interactuar con el sistema operativo
import os
#Importamos la función clear_output de la librería IPython.display para poder limpiar la salida de la celda
from IPython.display import clear_output
#Importamos la librería random para poder generar números aleatorios 
import random
#Importamos la librería matplotlib para poder realizar gráficas 
import matplotlib.pyplot as plt
#Importamos la librería pickle para poder guardar los datos en un archivo 
import pickle
#Importamos la librería time para poder utilizar la función sleep
import time
#Importamos la librería pandas para poder trabajar con dataframes 
import pandas as pd
from matplotlib import animation
import os
from IPython.display import clear_output
import random
import matplotlib.pyplot as plt
import pickle
import time

## Creacion de funcion para graficar las recompensas acumuladas cada episodio
Esta Clase tien la funcion de graficar, el cual toma los parametros de premios por episodio, el numero de episodios:

In [37]:

def Graphic_Average_reward_per_alpha(reward_per_epsilon, alphas):
    plt.figure(figsize=(8, 5))
    for alpha in alphas:
        plt.plot(reward_per_epsilon[alpha], label=f'$\epsilon$ = {alpha}')
    plt.legend()
    plt.grid(True)
    plt.xlabel('Episodios')
    plt.ylabel('Recompensa media')
    plt.title('Recompensa media por episodio para diferentes valores de $\epsilon$')
    plt.show()

In [42]:
def run(episodes, is_training=True, render=False):
    env = gym.make('CliffWalking-v0', render_mode="human" if render else None)

    if(is_training):
        q = np.zeros((env.observation_space.n, env.action_space.n))
    else:
        with open('CliffWalking.pkl', 'rb') as f:
            q = pickle.load(f)

    learning_rate_a = 0.1
    discount_factor_g = 0.99
    epsilon = 1
    epsilon_decay_rate = 0.0005
    rng = np.random.default_rng()
    RENDER_AT_EPISODE = 10
    time_step=0
    rewards_per_episode = np.zeros(episodes)

    for i in range(episodes):
        if i%10_000 == 0:
            print(f"Currently on episode {i}")
        state = env.reset()
        terminated = False
        truncated = False
        episode_reward = 0

        while(not terminated and not truncated):
            if is_training and rng.random() < epsilon:
                action = env.action_space.sample() # escoje una accion aleatoria 'a = np.random.randint(5) + 1'
            else:
                action = np.argmax(q[state,:]) # escoje la mejor accion 'mejor_accion = max(q, key=q.get)'

            new_state, reward, terminated, truncated, info = env.step(action)

            if is_training:
                q[state,action] = q[state,action] + learning_rate_a * (
                    reward + discount_factor_g * np.max(q[new_state,:]) - q[state,action]
                )
            state = new_state
            episode_reward += reward
            time_step+=1
            if i%RENDER_AT_EPISODE == 0:
                clear_output(wait=True)
                env.render()

        epsilon = max(epsilon - epsilon_decay_rate, 0.01)
        rewards_per_episode[i] = episode_reward
        print(f"Episode {i}/{episodes}, Epsilon: {epsilon:.4f}, Reward: {episode_reward:.4f}")

    env.close()

    window_size = 1000
    sum_rewards = np.zeros(episodes)
    for t in range(episodes):
        sum_rewards[t] = np.sum(rewards_per_episode[max(0, t-window_size):(t+1)])
    
    print("Final average reward:", np.mean(sum_rewards[-1000:]))
    plt.plot(sum_rewards)
    plt.savefig('CliffWalking.png')

    if is_training:
        with open("CliffWalking.pkl", "wb") as f:
            pickle.dump(q, f)

    return sum_rewards

: 

In [41]:
if __name__ == "__main__":
    run(5000, is_training=True, render=False)

Currently on episode 0


TypeError: only integer scalar arrays can be converted to a scalar index